<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/main/template-report-lab-X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative methods**
**Marc Hétier**

# **Abstract**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [114]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [115]:
# Load neccessary modules.
# from google.colab import files
import numpy as np
from copy import deepcopy

# **Introduction**

# **Method**

## Problem 1 : Jacobi iteration for $Ax= b$
To solve the problem $Ax=b$ we often use preconditionner before applying fixed point iteration. It exists different types of preconditionner, and some of them are based on matrix splitting. It is the case for Jacobi iteration and Gauss Seidel iteration (cf Problem 2).
For the general cases, we write :
$$ A = A_1 + A_2 $$
 with $A_1$ invertible.
Then, solution for $Ax = b$ is equivalently a solution of
$$(I+A_1^{-1})x = A_1^{-1}b $$
Note that we gain something only if solving both problem $A_1 y = b$ and $ (I+A_1^{-1})x = y$ are easier than the original one. 

The fixed point iteration for this problem writes :
$$x^{k+1} = -A_1^{-1}A_2 x^{k} + A_1^{-1}b $$


For **Jacobi iteration** we use $A_1 = Diag(A)$. This leads to a simple composant wise iteration :

$$ \forall i, \;\; x_i^{k+1} = \dfrac{1}{a_{ii}} (b_i - \sum_{j \neq i}a_{ij}x_j^k) $$

To ensure convergence criteria, we can assume that $A$ is striclty diagonally dominant.

In [116]:
def Jacobi_ite(A, b, x_0, m):
    """
    Input : matrix A, diagonal dominant vector b, initial guess x_0 and number of iterations m
    Output : m th iteration of Jacobi iteration for problem Ax = b 
    """
    n = len(A)
    test = 1
    for i in range(n):
        summ = sum(abs(A[i,:]))
        if 2*abs(A[i,i]) <= summ:
            test = 0
            break
    if not test:
        print("A is not strictly diagonally dominant; method can not converge")

    x = deepcopy(x_0)
    cst_term = np.array([b[i]/A[i,i] for i in range(n)])

    for _ in range(m):
        scd_term = np.array([1/A[i,i] * sum([A[i,j]*x[j] for j in range(n) if j != i])\
                     for i in range(n)])
        x = cst_term - scd_term
    
    return x   

## Problem 2 : Gauss-Seidel iteration for $Ax=b$

As said before, this method also use splitting preconditionner but this time with $A_1$ equal at the lower triangular part of $A$. This leads to the following componant wise iteration :

$$ \forall i, \;\; x_i^{k+1} = \dfrac{1}{a_{ii}} (b_i - \sum_{j = 1}^{i-1}a_{ij}x_j^{k+1} - \sum_{j = i+1}^{n}a_{ij}x_j^{k}) $$


The method converges when $A$ is positive definite or strictly diagonally dominant.

In [117]:
def Gauss_Seidel_ite(A, b, x_0, m):
    """
    Input : matrix A, diagonal dominant vector b, initial guess x_0 and number of iterations m
    Output : m th iteration of Jacobi iteration for problem Ax = b 
    """
    n = len(A)
    test = 1
    for i in range(n):
        summ = sum(abs(A[i,:]))
        if 2*abs(A[i,i]) <= summ:
            test = 0
            break
    if not test:
        print("A is not strictly diagonally dominant; method can not converge")

    x = deepcopy(x_0)
    cst_term = np.array([b[i]/A[i,i] for i in range(n)])
    for _ in range(m):
        x_1 = np.zeros(n)
        for i in range(n):
            scd_term = sum([A[i,j]*x_1[j] for j in range(i)])/A[i,i]
            trd_term = sum([A[i,j]*x[j] for j in range(i+1, n)])/A[i,i]
            x_1[i] = cst_term[i] - scd_term - trd_term
        x = x_1
    
    return x   

## Problem 3 : Newton's method for scalar non linear function

In this problem, we use Newton's method to solve a scalar non linear equation $$f(x) = 0$$ where $f$ is a $C^1$ function over some interval $I$, with $f'(x) \neq 0$ for $x\in I$.

This method is an iterative one, based on the following sequence :
$$ x^{k+1} = x^k - \dfrac{f(x^k)}{f'(x^k)} $$

Geometrically, this means that $x^{k+1}$ corresponds to the zero value of the tangent line at point $x^k$.

In [118]:
def Newton_method(f, df, x, tol, ite = 500):
    """
    Input : a function f, its derivative df, an initial guess x
            and a tolerance criteria tol
    Output : the convergence point if the tolerance is reached, or
            point after ite=500 iterations
    """
    xk = x
    test = 0
    for _ in range(ite):
        f_xk = f(xk)
        if np.abs(f_xk) < tol:
            test = 1
            break

        df_xk = df(xk)
        if df_xk == 0:
            print("The derivative evaluated at xk = ", xk, "is null")
            return None

        xk = xk - f_xk/df_xk
    
    if np.abs(f_xk) < tol:
            test = 1

    if test:
        return xk
    else:
        print("Stopping criteria not reached, end after ", ite, "iterations")
        return xk      

## Problem 4 : GMRES method for Ax = b

GMRES (Generalized minimal residual) is a Krylov method for solving large problem $Ax = b$. It uses the following Arnoldi factorization, obtained for exemple using Gram-Schmidt procedure on the vectorial space $K_m(A, b) = span(b, Ab, \ldots, A^{m-1}b)$ :
$$ A Q_m = Q_{m+1} H_m$$
with $Q_m$ othonormal matrix of size $n\times m$, $H_m$ Hessenberg matrix.

The GMRES method consits to minimizing the 2-norm of the residual over $K_m(A,b)$ *ie* :
$$x_m = \underset{x \in K_m(A,b)}{\text{argmin}} \Vert Ax -b \Vert $$

Using the fact that $K_m(A,b) = span(Q_m)$ and that $b = \Vert b \Vert Q_{m+1} e_1$, we have :

\begin{align*}
x_m &= Q_m \times \underset{x \in \mathbb{R}^m}{\text{argmin}} \Vert AQ_mx - \Vert b \Vert Q_{m+1} e_1 \Vert \\
x_m &= Q_m \times \underset{x \in \mathbb{R}^m}{\text{argmin}} \Vert Q_{m+1}(H_m x - \Vert b \Vert e_1) \Vert \\
x_m &= Q_m \times \underset{x \in \mathbb{R}^m}{\text{argmin}} \Vert H_m x - \Vert b \Vert e_1 \Vert
\end{align*}

This last problem is only of size $(m+1) \times m$ with $m << n$, while the initial one was of size $n\times n$.

In [119]:
def Gram_schmidt(Q, a):
    """
    Input : an orthonormal matrix Q, a vector x
    Output : result of Gram-Schmidt procedure applied to (Q, a).
    """
    _, M = Q.shape
    q = deepcopy(a)
    coord = np.zeros(M)
    for m in range(M):
        ps = np.dot(Q[:,m], q)
        q = q - ps*Q[:,m]
        coord[m] = ps
    
    norm = np.linalg.norm(q)
    return q, coord, norm

In [120]:
def GMRES(A, b, m):
    """
    Input : matrix A, vector b, range of approximation m
    Output : result of GMRES applied to Ax = b, approximation at level m
    """
    n = len(A)

    ## Compute the Arnoldi factorization
    norm_b = np.linalg.norm(b)
    a = b/norm_b

    Q_m = np.zeros((n, m+1))
    Q_m[:,0] = a
    H_m = np.zeros((m+1, m))

    for ind in range(0,m):
        a = A@Q_m[:,ind]
        q, coord, norm = Gram_schmidt(Q_m[:,0:ind+1], a)
        Q_m[:,ind+1] = q/norm
        H_m[0:ind+1, ind] = coord
        H_m[ind+1, ind] = norm
    
    ## Solve reduce problem
    e_1 = np.zeros((m+1, 1))
    e_1[0] = norm_b
    x = np.linalg.lstsq(H_m, e_1, rcond=None)[0]
    x = Q_m[:,0:m]@x

    return x

# **Results**

## Test for problem 1 and 2
We will test the Jacobi and Gauss-Seidel methods using the same problem. We first construct a matrix $A$, striclty diagonally dominant, and then copute the residual and the difference with an exact solution, given by the built in function of numpy.linalg.

In [121]:
A = np.reshape(np.arange(10, 135, 5), (5,5))
for i in range(5):
    A[i,i] = sum(A[i,j] for j in range(5))
b = np.ones(5)

x = np.linalg.solve(A, b)

x_J = Jacobi_ite(A, b, np.zeros(5), 10)
res_J = np.linalg.norm(A@x_J - b)
diff_J = np.linalg.norm(x_J-x)
print("Residual norm for Jacobi method, after 10 iterations : ", res_J, "difference with the exact sol : ", diff_J)
x_J = Jacobi_ite(A, b, np.zeros(5), 50)
res_J = np.linalg.norm(A@x_J - b)
diff_J = np.linalg.norm(x_J-x)
print("Residual norm for Jacobi method, after 50 iterations : ", res_J, "difference with the exact sol : ", diff_J)

x_GS = Gauss_Seidel_ite(A, b, np.zeros(5), 5)
res_GS = np.linalg.norm(A@x_GS - b)
diff_GS = np.linalg.norm(x_GS-x)
print("\n Residual norm for Gauss-Seidel method, after 5 iterations : ", res_GS, "difference with the exact sol : ", diff_GS)
x_GS = Gauss_Seidel_ite(A, b, np.zeros(5), 10)
res_GS = np.linalg.norm(A@x_GS - b)
diff_GS = np.linalg.norm(x_GS-x)
print("Residual norm for Gauss-Seidel method, after 10 iterations : ", res_GS, "difference with the exact sol : ", diff_GS)



Residual norm for Jacobi method, after 10 iterations :  0.44120422056490405 difference with the exact sol :  0.000636541423002796
Residual norm for Jacobi method, after 50 iterations :  0.000125925978590891 difference with the exact sol :  1.8167720306517036e-07

 Residual norm for Gauss-Seidel method, after 5 iterations :  0.00013346445496273884 difference with the exact sol :  8.146161408684108e-07
Residual norm for Gauss-Seidel method, after 10 iterations :  3.360420361660094e-08 difference with the exact sol :  1.4429833711263496e-10


Both method converges in few iterations, but Gauss-Seidel method is much faster than Jacobi method (ten times less iteration are required to obtain the same residual norm).

## Test for Problem 3 :

We will test the Newton's method on the function $f(x) = x^2 -1$, using a starting point at $x=0.5$, and at $x=-0.5$.

In [122]:
f = lambda x:x**2-1
df = lambda x:2*x

x_1 = 0.5
x_2 = -0.5

x_f1 = Newton_method(f, df, x_1, 10e-6)

print("Starting point : ", 0.5)
print("Residual convergence : ", np.abs(f(x_f1)))
print("Difference with expected root 1 : ", np.abs(x_f1-1))

x_f2 = Newton_method(f, df, x_2, 10e-6)

print("\nStarting point : ", -0.5)
print("Residual convergence : ", np.abs(f(x_f2)))
print("Difference with expected root -1 : ", np.abs(x_f2+1))

Starting point :  0.5
Residual convergence :  9.292229696811205e-08
Difference with expected root 1 :  4.6461147373833e-08

Starting point :  -0.5
Residual convergence :  9.292229696811205e-08
Difference with expected root -1 :  4.6461147373833e-08


The Newton method converge with a quadratic rate since the function $f$ is $C^2$. Moreover, we can see the perfect symmetry between the residual/difference of the two starting points. This was expected because of the problem's symmetry.

## Test for problem 4

Finally, we can test the GMRES function using a medium size matrix $A$:

In [123]:
def create_tridiag(size):
    d = np.diag([5 for _ in range(size)], 0)
    sd = [-1 for _ in range(size-1)]
    ud = np.diag(sd, 1)
    ld = np.diag(sd, 1)
    return d + ud + ld

A = create_tridiag(200)
b = np.ones(200)

x = GMRES(A, b, 10)
print("Residual after 10 iterations :", np.linalg.norm(A@x - b))

x = GMRES(A, b, 25)
print("Residual after 25 iterations :", np.linalg.norm(A@x - b))

x = GMRES(A, b, 50)
print("Residual after 50 iterations :", np.linalg.norm(A@x - b))

Residual after 10 iterations : 0.002251400858594942
Residual after 25 iterations : 2.4161676951621126e-09
Residual after 50 iterations : 6.388347155134135e-13


We can see that after 50 iterations, the method gives a solution which almost reach the machine precision. Then GMRES seems to be well implemented.
Note that I have change the diagonal of A from 2 to 5 to get better convergence rate (which is highly connected with the eigenvalue of A)

# **Discussion**

All the methods implemented wok well, and it is interesting to see their different convergence rate. Then, depending on $A$, one should privilige one method in particular, instead of the others.